# Day 2 - Lab 2: Documenting Key Decisions with ADRs

**Objective:** Use an LLM as a research assistant to compare technical options and synthesize the findings into a formal, version-controlled Architectural Decision Record (ADR).

**Estimated Time:** 60 minutes

**Introduction:**
Great architectural decisions are based on research and trade-offs. A critical practice for healthy, long-lived projects is documenting *why* these decisions were made. In this lab, you will use an LLM to research a key technical choice for our application and then generate a formal ADR to record that decision for the future.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll start by ensuring our environment is ready and adding the standard pathing solution to reliably import our `utils.py` helper.

**Model Selection:**
For research and synthesis tasks, models with large context windows and strong reasoning abilities are ideal. `gpt-4.1`, `gemini-2.5-pro`, or `meta-llama/Llama-3.3-70B-Instruct` would be excellent choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the ADR template.
- `save_artifact()`: To save the generated ADR template and the final ADR.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

2025-10-02 11:58:48,189 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4.1 latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): The ADR Template

**Task:** A good ADR follows a consistent format. Your first task is to prompt an LLM to generate a clean, reusable ADR template in markdown.

**Instructions:**
1.  Write a prompt that asks the LLM to generate a markdown template for an Architectural Decision Record.
2.  The template should include sections for: `Title`, `Status` (e.g., Proposed, Accepted, Deprecated), `Context` (the problem or forces at play), `Decision` (the chosen solution), and `Consequences` (the positive and negative results of the decision).
3.  Save the generated template to `templates/adr_template.md`.

In [2]:
# TODO: Write a prompt to generate a markdown ADR template.
adr_template_prompt = "Generate a detailed markdown template for an Architecture Decision Record (ADR). The template should include \
    sections for Title, Status: (Proposed, Accepted, Deprecated), Context: (the problem or situation that prompted the decision),  \
    Consequences: (the outcomes of the decision). Provide brief descriptions for each section to guide the user on \
    what information to include. Format the output as a markdown document and save in fictional-waffle/artifacts/adr_template.md"

adr_template_prompt = prompt_enhancer(adr_template_prompt)

print("--- Generating ADR Template ---")
adr_template_content = get_completion(adr_template_prompt, client, model_name, api_provider)
print(adr_template_content)

# Save the artifact
if adr_template_content:
    save_artifact(adr_template_content, "fictional-waffle/artifacts/adr_template.md", overwrite=True)

2025-10-02 12:07:44,118 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Generating ADR Template ---
# fictional-waffle/artifacts/adr_template.md
```markdown
# <Title>
*Short, descriptive name for this architectural decision.*

## Status
*Proposed / Accepted / Deprecated / Superseded (select one and date).*

## Context
*Describe the problem, constraints, and any forces that influenced this decision.*

## Decision
*State the option chosen and give rationale for choosing it over alternatives.*

## Consequences
*List the positive and negative outcomes that result from this decision.*
```


### Challenge 2 (Intermediate): AI-Assisted Research

**Task:** Use the LLM to perform unbiased research on a key technical decision for our project: choosing a database for semantic search.

**Instructions:**
1.  Write a prompt instructing the LLM to perform a technical comparison.
2.  Ask it to compare and contrast two technical options: **"Using PostgreSQL with the `pgvector` extension"** versus **"Using a specialized vector database like ChromaDB or FAISS"**.
3.  The prompt should ask for a balanced view for the specific use case of our new hire onboarding tool.
4.  Store the output in a variable for the next step.

> **Tip:** To get a balanced comparison, explicitly ask the LLM to 'act as an unbiased research assistant' and to list the 'pros and cons for each approach.' This prevents the model from simply recommending the more popular option and encourages a more critical analysis.

In [3]:
# TODO: Write a prompt to research database options.
db_research_prompt = "Research and compare two different database solutions ('Using PostgreSQL with the pgvector extension' and \
                      'Using a specialized vector database like ChromaDB or FAISS') for a web application that manages user \
                      onboarding tasks. Consider factors such as scalability, ease of use, community support, and integration with \
                      Python applications. Act as an unbiased research assistant to provide a summary of the pros and cons of each \
                      option and recommend the best choice for this use case."

db_research_prompt = prompt_enhancer(db_research_prompt)

print("--- Researching Database Options ---")
db_research_output = get_completion(db_research_prompt, client, model_name, api_provider)
print(db_research_output)

2025-10-02 12:08:48,323 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Researching Database Options ---
## Comparative Analysis

| Factor | PostgreSQL + pgvector | ChromaDB / FAISS |
|--------|----------------------|------------------|
| **Scalability** | Vertical scaling is robust; can handle moderate vector workloads (<10M vectors) well. Horizontal scaling (sharding, replication) is mature for relational data, but vector operations may degrade in high-concurrency or high-volume scenarios. Indexing (IVFFlat, HNSW) is available, but ANN search performance lags behind specialized engines for very large datasets. | Designed for high-throughput vector workloads; FAISS is optimized for billion-scale vectors in-memory, while ChromaDB is Python-native and can scale horizontally (though production-grade clustering is still maturing). Native support for distributed architectures (especially in commercial managed offerings). Relational joins and transactional integrity are limited or absent. |
| **Ease of Use** | Familiar setup for teams using Django/FastAPI; 

### Challenge 3 (Advanced): Synthesizing the ADR

**Task:** Provide the LLM with your research from the previous step and have it formally document the decision.

**Instructions:**
1.  Load the `adr_template.md` you created in the first challenge.
2.  Create a new prompt instructing the LLM to act as a Staff Engineer.
3.  Provide the `db_research_output` as context.
4.  Instruct the LLM to populate the ADR template, formally documenting the decision to **use PostgreSQL with pgvector** and justifying the choice based on the synthesized pros and cons.
5.  Save the final, completed ADR as `artifacts/adr_001_database_choice.md`.

In [4]:
adr_template = load_artifact("fictional-waffle/artifacts/adr_template.md")

# TODO: Write a prompt to synthesize the final ADR.
synthesis_prompt = f"Acting as a staff engineer, use the context of the {adr_template} and the previous research on database options \
                    in {db_research_output} to synthesize a comprehensive Architecture Decision Record (ADR). \
                    Ensure that the ADR clearly states the decision made, the context leading to the decision, and \
                    the consequences of the decision. Format the output as a markdown document."

print("--- Synthesizing Final ADR ---")
if adr_template and 'db_research_output' in locals() and db_research_output:
    final_adr = get_completion(synthesis_prompt, client, model_name, api_provider)
    print(final_adr)
    save_artifact(final_adr, "fictional-waffle/artifacts/adr_001_database_choice.md")
else:
    print("Skipping ADR synthesis because template or research is missing.")

--- Synthesizing Final ADR ---
```markdown
# Vector Database Selection for Semantic Search in SaaS Web Application

## Status
Accepted – 2024-06-10

## Context

As part of enhancing our SaaS web application, we require a solution to support semantic search capabilities using vector similarity search. The application must manage both traditional relational data (e.g., user onboarding, workflow management) and moderate-scale vector data (e.g., text embeddings for search and recommendations). The engineering team predominantly uses Python with Django/FastAPI, and operational simplicity, reliability, and maintainability are key priorities.

We evaluated two primary approaches for storing and querying vector data:

1. **PostgreSQL with pgvector extension:** Enables native storage and search of vector embeddings within a standard PostgreSQL database, supporting both relational and vector use cases in a unified system.
2. **Specialized vector databases (ChromaDB, FAISS):** Designed for high-t

## Lab Conclusion

Well done! You have used an LLM to automate a complex but critical part of the architectural process. You leveraged its vast knowledge base for research and then used it again for synthesis, turning raw analysis into a formal, structured document. This `adr_001_database_choice.md` file now serves as a permanent, valuable record for anyone who works on this project in the future.

> **Key Takeaway:** The pattern of **Research -> Synthesize -> Format** is a powerful workflow. You can use an LLM to gather unstructured information and then use it again to pour that information into a structured template, creating high-quality, consistent documentation with minimal effort.